In [1]:
import numpy as np
import pandas as pd
#import asyncio
#import aiohttp
import json
import csv
import requests
import json
import time
from datetime import datetime
from datetime import date
import numpy as np
import pandas as pd

In [2]:


def get_markets(api = "cryptowatch", active = True):
    """get markets available in API"""
    if api == "cryptowatch":
        try:
            r = requests.get("https://api.cryptowat.ch/markets").json()['result']
            if (active):
                exchanges = {item['exchange'] for item in r if item['active'] is True}
            else:
                exchanges = {item['exchange'] for item in r}
        except:
            print("Error: disfunctional API from Cryptowatch")
            

    return exchanges

def get_pairs(api = "cryptowatch", active = True):
    """get pairs available in API"""
    if api == "cryptowatch":
        try:
            r = requests.get("https://api.cryptowat.ch/markets").json()['result']
            if (active):
                pairs = {item['pair'] for item in r if item['active'] is True}
            else:
                pairs = {item['pair'] for item in r}
        except:
            print("Error: disfunctional API from Cryptowatch")
    return pairs



In [3]:
m = get_markets()

In [4]:
def list_markets_containing_pair(pair = 'btcusd'):
    try:
        r = requests.get("https://api.cryptowat.ch/markets").json()['result']
        exchanges = {item['exchange'] for item in r if (item['active'] is True and item['pair'] == pair)}
        return exchanges
    except:
        print('error')

def list_pairs(pairs=get_pairs(), p1='btc'):
    pairs = list(pairs)
    L = []
    for pair in pairs:
        if p1 == pair[:len(p1)]:
            L.append(pair)
    return L

def list_pairs_in_market(market):
    r = requests.get("https://api.cryptowat.ch/markets/"+market).json()['result']
    market_pairs = {item['pair'] for item in r if item['active'] is True}
    return market_pairs

def list_markets_containing_pair_dumb(pair = 'btcusd', markets_list = list(get_markets())  ):
    for mkt in markets_list:
        if pair in list_pairs_in_market(mkt):
            print(mkt)


In [5]:
def get_ohlc(market, pair, period = 1440, api = "cryptowatch", start = "", end = "", local_timezone = True):
    """get OHLC prices available in API"""
    period *= 60
    period = str(period)
    params = []
    url = "https://api.cryptowat.ch/markets/" + market + "/" + pair + "/ohlc"
#    print(url)
    if (end == ""):
        before = ""
    else:
        before = time.mktime(time.strptime(end, "%d %m %y %H %M"))
    if (start == ""):
        after = start
    else:
        after = time.mktime(time.strptime(start, "%d %m %y %H %M"))
    if (not before or not after):
        prms = [('before', before), ('after', after)]
    else:
        prms = {'before': int(before) , 'after': int(after) }

    
    if api == "cryptowatch":
        try:
            r = requests.get(url, params=prms).json()['result'][period]
            if (local_timezone):            
                a = pd.DataFrame(r, columns = ['date(local)','open','high','low','close','volume','neg'])
                a[a.columns[0]] = pd.to_datetime(a[a.columns[0]] - time.timezone, unit = 's')
                a = a.set_index('date(local)')
 
                return a
            else:
                a = pd.DataFrame(r, columns = ['date(utc)','open','high','low','close','volume','neg'])
                a[a.columns[0]] = pd.to_datetime(a[a.columns[0]], unit = 's')
                a = a.set_index('date(utc)')

                return a               
                

        except:
            print("Error: disfunctional API from Cryptowatch")
            
#columns = ['period','open','high','low','close','volume','neg']    

In [27]:
def compare_pair_across_markets(pair = 'btcusd'):
    a = list_markets_containing_pair(pair)
    print(a)
    c = {}
    for m in list(a):
        print(m, pair)
        c[m] = get_ohlc(m,pair)
    return c

In [28]:
b = list_markets_containing_pair('btcusd')
#print(b)
c = compare_pair_across_markets()
#print(c)
#s1 = "14 07 18 00 00"
#e1 = "15 07 18 00 00"
#print(time.timezone)
#a = get_ohlc("bitfinex", "btcusd", period = 240, start = s1, end = e1)
#a = get_ohlc("bitfinex", "btcusd")
#print(c)

{'gdax', 'bitflyer', 'bitstamp', 'quoine', 'cexio', 'gemini', 'bitfinex', 'kraken', 'okcoin', 'bitsquare', 'quadriga', 'bittrex', 'bitbay', 'wex'}
gdax btcusd
bitflyer btcusd
bitstamp btcusd
quoine btcusd
cexio btcusd
gemini btcusd
bitfinex btcusd
kraken btcusd
okcoin btcusd
bitsquare btcusd
quadriga btcusd
bittrex btcusd
bitbay btcusd
wex btcusd
{'gdax':                         open     high      low    close        volume  \
date(local)                                                             
2017-03-12 21:00:00  1188.28  1244.00  1150.00  1235.58   6727.292500   
2017-03-13 21:00:00  1235.27  1249.00  1207.98  1245.49   9512.914000   
2017-03-14 21:00:00  1245.50  1262.00  1213.55  1247.42   7463.764600   
2017-03-15 21:00:00  1247.43  1264.55  1240.00  1263.00   7422.759000   
2017-03-16 21:00:00  1263.55  1277.27  1117.77  1175.11  16616.162000   
2017-03-17 21:00:00  1175.10  1175.11  1065.74  1069.57  19295.305000   
2017-03-18 21:00:00  1068.78  1096.00   919.44   970.00  2

In [36]:
import os

In [41]:

def store_pair_data(pair, pair_data):
    try:
        os.mkdir(pair)
    except:
        os.rmdir(pair)
        os.mkdir(pair)
        
    
for key in list(c.keys()):
    i = i + 1
    c[key].to_csv("data/"+key+".csv")
    #c1.to_csv()


In [40]:
store_pair_data('btcusd')

FileExistsError: [Errno 17] File exists: 'btcusd'

In [ ]:
def get_daily_price(asset="btc", api="coinmetrics", data_type="price(usd)", start=0, end=time.time()):
    if (api == "coinmetrics"):
        try:
            url =  "https://coinmetrics.io/api/v1/get_asset_data_for_time_range/"+asset+"/"+data_type
            url = url + "/"+str(int(start)) +"/"+str(int(end))
            r = requests.get(url).json()['result']
            #print(type(r))
            #print(r)
            a = pd.DataFrame(r, columns = ['date(utc)',str(asset)+":"+str(data_type)])
            print(url)
            a[a.columns[0]] = pd.to_datetime(a[a.columns[0]], unit = 's')

            return a
        except:
            print("Error: disfunctional API from Coinmetrics")            
            
    else:
        return 0



In [ ]:
a = get_daily_price()
print(time.time())
a.tail()


# Terça-Feira

- finalizar get_ohlc() (from datetime to timestamp)

- atualizar get_pairs() colocando um parâmetro active = True para retornar só pares ativos

```Python 
pairs = {item['pair'] for item in r if item['active'] is True} ```

- O mesmo para mercados

